### IMPORT PACKAGES

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

UsageError: Line magic function `%` not found.


In [51]:
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData

In [52]:
Username = 'admin'
Password = 'amazing_people'
Host = 'alphacode-explore.ccwgqdqrrmvt.eu-west-1.rds.amazonaws.com'
Port = '1433'
Database = 'Michael'

### CREATING CONNECTION

In [53]:
connection = create_engine(
    'mssql+pymssql://' +
    Username + ':' + Password + '@' + Host + ':' + Port + '/' + Database
)

### SQL QUERY

In [54]:
sql_query = "Select * from claims_data"

### CREATING A DATAFRAME

In [55]:
df = pd.read_sql_query(sql_query, connection)

In [56]:
df.head()

,age,sex,bmi,steps,children,smoker,region,insurance_claim,claim_amount
0,19,female,27.9,3009,0,yes,southwest,yes,16884.924
1,18,male,33.77,3008,1,no,southeast,yes,1725.5523
2,28,male,33,3009,3,no,southeast,no,0
3,33,male,22.705,10009,0,no,northwest,no,0
4,32,male,28.88,8010,0,no,northwest,yes,3866.8552


### PRE-PROCESSING

### FEATURES/LABELS SPLIT

In [57]:
# Lables
y = df['insurance_claim']

# Features
X = df.drop(['insurance_claim', 'claim_amount'], axis=1)

In [58]:
X.shape

(1338, 7)

### TRANSFORMING THE FEATURES

In [59]:
X_transformed = pd.get_dummies(X, drop_first=True)

### TRAIN/TEST SPLIT

### IMPORT TRAIN-TEST-SPLIT METHOD

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=47)

### TRAINING MODEL

### Rescaling features

In [62]:
from sklearn.preprocessing import StandardScaler

In [63]:
scaler = StandardScaler()

### Standardizing features

In [64]:
X_scaled = scaler.fit_transform(X_transformed)

In [65]:
X_standardize = pd.DataFrame(X_scaled,columns=X_transformed.columns)
X_standardize.head()

,age_19,age_20,age_21,age_22,age_23,age_24,age_25,age_26,age_27,age_28,...,steps_8010,children_1,children_2,children_3,children_4,children_5,smoker_yes,region_northwest,region_southeast,region_southwest
0,4.321628,-0.148843,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,...,-0.161473,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,1.970587,-0.566418,-0.611324,1.765481
1,-0.231394,-0.148843,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,...,-0.161473,1.769076,-0.467525,-0.364607,-0.137987,-0.116775,-0.507463,-0.566418,1.635795,-0.566418
2,-0.231394,-0.148843,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,6.840008,...,-0.161473,-0.565267,-0.467525,2.742680,-0.137987,-0.116775,-0.507463,-0.566418,1.635795,-0.566418
3,-0.231394,-0.148843,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,...,-0.161473,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.507463,1.765481,-0.611324,-0.566418
4,-0.231394,-0.148843,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,-0.146199,...,6.192975,-0.565267,-0.467525,-0.364607,-0.137987,-0.116775,-0.507463,1.765481,-0.611324,-0.566418


### Training

In [66]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [67]:
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier(random_state=47)

In [68]:
knn.fit(X_train, y_train)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=47, splitter='best')

### EVALUATE

In [69]:
pred_knn = knn.predict(X_test)
pred_tree = tree.predict(X_test)

### Testing

In [70]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

Confusion matrix

In [71]:
labels = ['No', 'Yes']

print('Confusion Matrix')
pd.DataFrame(data=confusion_matrix(y_test, pred_knn), index=labels, columns=labels)

Confusion Matrix


,No,Yes
No,57,45
Yes,24,142


In [72]:
labels = ['No', 'Yes']

print('Decision Tree - Confusion Matrix')
pd.DataFrame(data=confusion_matrix(y_test, pred_tree), index=labels, columns=labels)

Decision Tree - Confusion Matrix


,No,Yes
No,80,22
Yes,13,153


Classification report

In [73]:
print('Classification Report')
print(classification_report(y_test, pred_knn, target_names=['No', 'Yes']))

Classification Report
              precision    recall  f1-score   support

          No       0.70      0.56      0.62       102
         Yes       0.76      0.86      0.80       166

    accuracy                           0.74       268
   macro avg       0.73      0.71      0.71       268
weighted avg       0.74      0.74      0.74       268



In [74]:
print('Decision Tree - Classification Report')
print(classification_report(y_test, pred_tree, target_names=['No', 'Yes']))

Decision Tree - Classification Report
              precision    recall  f1-score   support

          No       0.86      0.78      0.82       102
         Yes       0.87      0.92      0.90       166

    accuracy                           0.87       268
   macro avg       0.87      0.85      0.86       268
weighted avg       0.87      0.87      0.87       268



### TUNING PARAMETERS

## Train

In [75]:
knn_2 = KNeighborsClassifier(n_neighbors=2)
knn_4 = KNeighborsClassifier(n_neighbors=4)
knn_15 = KNeighborsClassifier(n_neighbors=15)

In [76]:
tree_1 = DecisionTreeClassifier(max_depth=2, min_samples_leaf=5, random_state=50)
tree_2 = DecisionTreeClassifier(max_depth=5, min_samples_leaf=4, random_state=50)
tree_3 = DecisionTreeClassifier(max_depth=7, min_samples_leaf=3, random_state=50)

In [77]:
knn_2.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [78]:
tree_1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=50, splitter='best')

In [79]:
knn_4.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [80]:
tree_2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=50, splitter='best')

In [81]:
knn_15.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=15, p=2,
                     weights='uniform')

In [82]:
tree_3.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=50, splitter='best')

### PREDICT

In [83]:
pred_knn_2 = knn_2.predict(X_test)
pred_knn_4 = knn_4.predict(X_test)
pred_knn_15 = knn_15.predict(X_test)

In [84]:
pred_tree_1 = tree_1.predict(X_test)
pred_tree_2 = tree_2.predict(X_test)
pred_tree_3 = tree_3.predict(X_test)

# TEST

Confusion Matrix

In [85]:
labels = ['No', 'Yes']

pred = {'k=2':pred_knn_2, 'k=4':pred_knn_4, 'k=15':pred_knn_15}

for k,v in pred.items():
    print('{}'.format(k))
    print(pd.DataFrame(data=confusion_matrix(y_test, v), index=labels, columns=labels))
    print('\n')

k=2
     No  Yes
No   85   17
Yes  63  103


k=4
     No  Yes
No   73   29
Yes  42  124


k=15
     No  Yes
No   69   33
Yes  20  146




In [86]:
labels = ['No', 'Yes']

pred = {'max_depth=2; min_samples_leaf=5':pred_tree_1, 
        'max_depth=5; min_samples_leaf=4':pred_tree_2, 
        'max_depth=7; min_samples_leaf=3':pred_tree_3}

for k,v in pred.items():
    print('{}'.format(k))
    print(pd.DataFrame(data=confusion_matrix(y_test, v), index=labels, columns=labels))
    print('\n')

max_depth=2; min_samples_leaf=5
     No  Yes
No   19   83
Yes   6  160


max_depth=5; min_samples_leaf=4
     No  Yes
No   78   24
Yes  34  132


max_depth=7; min_samples_leaf=3
     No  Yes
No   54   48
Yes  18  148




Classification report

In [87]:
pred = {'k=2':pred_knn_2, 'k=4':pred_knn_4, 'k=15':pred_knn_15}

for k,v in pred.items():
    print('{}'.format(k))
    print(classification_report(y_test, v, target_names=['No', 'Yes']))
    print('\n')

k=2
              precision    recall  f1-score   support

          No       0.57      0.83      0.68       102
         Yes       0.86      0.62      0.72       166

    accuracy                           0.70       268
   macro avg       0.72      0.73      0.70       268
weighted avg       0.75      0.70      0.70       268



k=4
              precision    recall  f1-score   support

          No       0.63      0.72      0.67       102
         Yes       0.81      0.75      0.78       166

    accuracy                           0.74       268
   macro avg       0.72      0.73      0.73       268
weighted avg       0.74      0.74      0.74       268



k=15
              precision    recall  f1-score   support

          No       0.78      0.68      0.72       102
         Yes       0.82      0.88      0.85       166

    accuracy                           0.80       268
   macro avg       0.80      0.78      0.78       268
weighted avg       0.80      0.80      0.80       268





In [88]:
pred = {'max_depth=2; min_samples_leaf=5':pred_tree_1, 
        'max_depth=5; min_samples_leaf=4':pred_tree_2, 
        'max_depth=7; min_samples_leaf=3':pred_tree_3}

for k,v in pred.items():
    print('{}'.format(k))
    print(classification_report(y_test, v, target_names=['No', 'Yes']))
    print('\n')

max_depth=2; min_samples_leaf=5
              precision    recall  f1-score   support

          No       0.76      0.19      0.30       102
         Yes       0.66      0.96      0.78       166

    accuracy                           0.67       268
   macro avg       0.71      0.58      0.54       268
weighted avg       0.70      0.67      0.60       268



max_depth=5; min_samples_leaf=4
              precision    recall  f1-score   support

          No       0.70      0.76      0.73       102
         Yes       0.85      0.80      0.82       166

    accuracy                           0.78       268
   macro avg       0.77      0.78      0.77       268
weighted avg       0.79      0.78      0.79       268



max_depth=7; min_samples_leaf=3
              precision    recall  f1-score   support

          No       0.75      0.53      0.62       102
         Yes       0.76      0.89      0.82       166

    accuracy                           0.75       268
   macro avg       0.75     

### DECISION TREE HAS THE BEST SCORE

### CREATING A DATAFRAME

In [193]:
sql_query_claims = "Select * from claims_data"
sql_query_test = "Select * from test_set"

df_claims = pd.read_sql_query(sql_query_claims, connection)
df_test = pd.read_sql_query(sql_query_test, connection)

df_claims_no_labels = df_claims.drop(['insurance_claim', 'claim_amount'], axis=1)
df_test_no_index = df_test.drop(['row_id'], axis=1)

df_combined = pd.concat([df_claims_no_labels, df_test_no_index],sort=False).reset_index(drop=True)

### Pre-processing

In [194]:
y = df_claims['insurance_claim']

X = df_claims.drop(['insurance_claim', 'claim_amount'], axis=1)

r = df_test['row_id']

### Transform features

In [195]:
combined_transformed = pd.get_dummies(df_combined, drop_first=True)

X_transformed = combined_transformed.iloc[:len(y), :]
test_transformed = combined_transformed.iloc[len(y):, :]

### Train/Test Split

In [197]:

from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=47)

### TRAINING

In [198]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=50)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=50, splitter='best')

### PREDICTING

In [199]:
pred_tree = tree.predict(X_test)

### TESTING

In [200]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

labels = ['No', 'Yes']

CONFUSION MATRIX

In [202]:
print('Decision Tree Confusion Matrix')
pd.DataFrame(data=confusion_matrix(y_test, pred_tree), index=labels, columns=labels)

Decision Tree Confusion Matrix


,No,Yes
No,80,22
Yes,13,153


### Classification report

In [203]:
from sklearn.metrics import classification_report

In [204]:
print('Decision Tree - Classification Report')
print(classification_report(y_test, pred_tree, target_names=['No', 'Yes']))

Decision Tree - Classification Report
              precision    recall  f1-score   support

          No       0.86      0.78      0.82       102
         Yes       0.87      0.92      0.90       166

    accuracy                           0.87       268
   macro avg       0.87      0.85      0.86       268
weighted avg       0.87      0.87      0.87       268



### TUNNING

In [205]:
tree_0 = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5, random_state=50)

tree_1 = DecisionTreeClassifier(max_depth=2, min_samples_leaf=5, random_state=50)

tree_2 = DecisionTreeClassifier(max_depth=5, min_samples_leaf=4, random_state=50)

tree_3 = DecisionTreeClassifier(max_depth=7, min_samples_leaf=3, random_state=50)

tree_0.fit(X_train, y_train)
tree_1.fit(X_train, y_train)
tree_2.fit(X_train, y_train)
tree_3.fit(X_train, y_train)

pred_tree_0 = tree_1.predict(X_test)
pred_tree_1 = tree_1.predict(X_test)
pred_tree_2 = tree_2.predict(X_test)
pred_tree_3 = tree_3.predict(X_test)

pred = {'max_depth=5; min_samples_leaf=5':pred_tree_0, 
        'max_depth=2; min_samples_leaf=5':pred_tree_1, 
        'max_depth=5; min_samples_leaf=4':pred_tree_2, 
        'max_depth=7; min_samples_leaf=3':pred_tree_3}

for k,v in pred.items():
    print('{}'.format(k))
    print(pd.DataFrame(data=confusion_matrix(y_test, v), index=labels, columns=labels))
    print('\n')

pred = {'max_depth=5; min_samples_leaf=5':pred_tree_0, 
        'max_depth=2; min_samples_leaf=5':pred_tree_1, 
        'max_depth=5; min_samples_leaf=4':pred_tree_2, 
        'max_depth=7; min_samples_leaf=3':pred_tree_3}

for k,v in pred.items():
    print('{}'.format(k))
    print(classification_report(y_test, v, target_names=['No', 'Yes']))
    print('\n')

max_depth=5; min_samples_leaf=5
     No  Yes
No   19   83
Yes   6  160


max_depth=2; min_samples_leaf=5
     No  Yes
No   19   83
Yes   6  160


max_depth=5; min_samples_leaf=4
     No  Yes
No   78   24
Yes  34  132


max_depth=7; min_samples_leaf=3
     No  Yes
No   53   49
Yes  18  148


max_depth=5; min_samples_leaf=5
              precision    recall  f1-score   support

          No       0.76      0.19      0.30       102
         Yes       0.66      0.96      0.78       166

    accuracy                           0.67       268
   macro avg       0.71      0.58      0.54       268
weighted avg       0.70      0.67      0.60       268



max_depth=2; min_samples_leaf=5
              precision    recall  f1-score   support

          No       0.76      0.19      0.30       102
         Yes       0.66      0.96      0.78       166

    accuracy                           0.67       268
   macro avg       0.71      0.58      0.54       268
weighted avg       0.70      0.67      0.60

In [207]:
test_pred_tree = tree.predict(test_transformed)

In [208]:
test_pred_tree

array(['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes',
       'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes',
       'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'yes',
       'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes',
       'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes',
       'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes',
       'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'yes',
       'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'no',
       'yes', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'yes',
       'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'yes', 'yes',
       'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes',
       'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no',
       'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'no', 'yes',
       'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', '

In [209]:
r

0        1
1        2
2        3
3        4
4        5
5        6
6        7
7        8
8        9
9       10
10      11
11      12
12      13
13      14
14      15
15      16
16      17
17      18
18      19
19      20
20      21
21      22
22      23
23      24
24      25
25      26
26      27
27      28
28      29
29      30
      ... 
270    271
271    272
272    273
273    274
274    275
275    276
276    277
277    278
278    279
279    280
280    281
281    282
282    283
283    284
284    285
285    286
286    287
287    288
288    289
289    290
290    291
291    292
292    293
293    294
294    295
295    296
296    297
297    298
298    299
299    300
Name: row_id, Length: 300, dtype: object

In [210]:
type(r)

pandas.core.series.Series

In [211]:
type(test_pred_tree)

numpy.ndarray

In [212]:
submission = pd.DataFrame({'row_id':r.values,'prediction': test_pred_tree})
submission.head(50)

,row_id,prediction
0,1,yes
1,2,yes
2,3,yes
3,4,yes
4,5,yes
5,6,yes
6,7,yes
7,8,yes
8,9,yes
9,10,yes


### Saving to SQL

In [213]:
submission.to_sql('classification_model_results', con=connection, if_exists='append', index=True,method='multi',chunksize=500)

In [214]:
connection.table_names()

['claims_data',
 'classification_model_results',
 'football_players',
 'sysdiagrams',
 'test_set',
 'test_set2']